In [ ]:
import pandas as pd
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPolygon
from scipy.spatial import cKDTree

# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [ ]:
gdf = gpd.read_file("laplata_cascourbano.kml", driver="KML")
print(gdf.head())


In [ ]:
# Chequear cuántas capas tiene el archivo kml
for layer in fiona.listlayers("laplata_cascourbano.kml"):
    print(layer)

In [ ]:
# Convertir el archivo kml a geojson
gdf.to_file("salida.geojson", driver='GeoJSON')

In [ ]:
# Plotear el archivo geojson
gdf = gpd.read_file("salida.geojson")
gdf.plot()

# Etapa 2 : Obtener datos de densidad poblacional de la ciudad de La Plata.

In [ ]:
# Dataset de densidad poblacional Argentina 2020 (1km resolución, ~4M puntos)
pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")

In [ ]:
# Función para extraer datos de una ciudad específica del dataset completo
def extract_city_data(city_name: str, bounds: dict) -> pd.DataFrame:

    df = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")
    
    city_data = df[
        (df['X'] >= bounds['x_min']) & 
        (df['X'] <= bounds['x_max']) & 
        (df['Y'] >= bounds['y_min']) & 
        (df['Y'] <= bounds['y_max'])
    ]
    
    # Guardar subset
    output_file = f"{city_name.lower().replace(' ', '_')}_population_2020.csv"
    city_data.to_csv(output_file, index=False)
    
    print("Filas extraídas:", f"{len(city_data):,}")
    
    return city_data

In [ ]:
# Coordenadas aproximadas para La Plata
la_plata_bounds = {
   'x_min': -58.2,
   'x_max': -57.7,
   'y_min': -35.2,
   'y_max': -34.7
}

# Extraer datos
la_plata_data = extract_city_data("La Plata", la_plata_bounds)


In [ ]:
# Leer el archivo CSV de densidad de población de La Plata
df = pd.read_csv('la_plata_population_2020.csv')
    
# Crear geometría de puntos usando X,Y como longitud,latitud
geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]
    

In [ ]:
# Crear GeoDataFrame
CRS_4326 = 4326
gdf_densidad= gpd.GeoDataFrame(df, geometry=geometry, crs=CRS_4326)
    
# Agregar columna de densidad con nombre descriptivo
gdf_densidad['densidad_poblacional_lp'] = gdf_densidad['Z']
gdf_densidad


In [ ]:
#gdf.to_crs() #cone esto cambio el sistema de referencia de coordenadas

In [ ]:
def export_to_geojson(gdf_densidad: gpd.GeoDataFrame, output_path:str = "la_plata_densidad.geojson") -> str:
        """
    Exporta un GeoDataFrame a formato GeoJSON
    
    Args:
        gdf_densidad: GeoDataFrame con datos de densidad
        output_path: Ruta de salida (default: "la_plata_densidad.geojson")
    
    Returns:
        str: Ruta del archivo exportado
    """
        
        gdf_densidad.to_file(output_path, driver='GeoJSON')
        print(f"Archivo exportado: {output_path}")
        return output_path

In [ ]:
export_to_geojson(gdf_densidad)

In [ ]:
# Visualización de la densidad poblacional de La Plata
fig, axes = plt.subplots(figsize=(6, 6))

# Gráfico de dispersión geográfica
scatter = axes.scatter(
    gdf_densidad['X'], gdf_densidad['Y'], c=gdf_densidad['Z'],
    cmap='inferno', alpha=0.6, s=10
)
axes.set_xlabel('Longitud')
axes.set_ylabel('Latitud')
axes.set_title('Distribución Geográfica de Densidad Poblacional de La Plata')
plt.colorbar(scatter, ax=axes, label='Densidad')

plt.tight_layout()
plt.savefig('densidad_la_plata.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Recortar el GeoDataFrame de densidad poblacional con el GeoDataFrame de La Plata

In [ ]:
def clip_density_to_urban_area(gdf_density: gpd.GeoDataFrame, gdf_geographical: gpd.GeoDataFrame) -> gpd.GeoDataFrame: 
    """
    Recorta los puntos de densidad que caen dentro del casco urbano de un límite geográfico dado.
    
    Args:
        gdf_density: GeoDataFrame con datos de densidad
        gdf_geographical: GeoDataFrame con límites geográficos

    Returns:
        gpd.GeoDataFrame: GeoDataFrame recortado a la zona urbana
    """
    # Usar spatial join para mantener solo puntos dentro del casco urbano
    points_in_casco = gpd.sjoin(gdf_density, gdf_geographical, how='inner', predicate='intersects')
    
    # Limpiar columnas duplicadas del join 
    points_in_casco = points_in_casco.drop(columns=[col for col in points_in_casco.columns if col.endswith('_right')]) 
    
    return points_in_casco

In [ ]:
casco_urbano = clip_density_to_urban_area(gdf_densidad, gdf)

In [ ]:
casco_urbano

In [ ]:
## Visualización simple 
#fig, ax = plt.subplots(figsize=(12, 10))
#
## Plotear el límite del casco urbano
#gdf.boundary.plot(ax=ax, color='red', linewidth=2, alpha=0.8)
#
## Plotear los puntos de densidad
#scatter = ax.scatter(casco_urbano['X'], casco_urbano['Y'], 
#                    c=casco_urbano['Z'], 
#                    cmap='viridis', 
#                    s=30, 
#                    alpha=0.7)
#
#plt.colorbar(scatter, label='Densidad Poblacional')
#ax.set_title('Casco Urbano - La Plata')
#ax.set_aspect('equal')
#plt.show()

# Etapa 3 : superposición de mapas de densidad y peligrosidad de inundación de la ciudad de La Plata (QGIS).

In [ ]:
# Verificaión de los sistemas de referencia espacial
gdf_mapa = gpd.read_file("salida.geojson")
print(gdf.crs)
print(gdf_mapa.crs)